# Como descobrir o número de de linhas nulas de cada coluna

In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [2]:
spark = SparkSession.builder \
    .master("local") \
    .config("spark.sql.autoBroadcastJoinThreshold", -1) \
    .config("spark.executor.memory", "500mb") \
    .appName("Ex8") \
    .getOrCreate()

22/07/05 14:14:58 WARN Utils: Your hostname, computador resolves to a loopback address: 127.0.1.1; using 10.0.0.135 instead (on interface wlp2s0)
22/07/05 14:14:58 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/07/05 14:14:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [5]:
df = spark.read.option('nullValue', 'null').csv('./data/emp.csv', header=True, inferSchema=True)

In [6]:
df.show()

+-----+------+---------+----+----------+----+----+------+------------+
|EMPNO| ENAME|      JOB| MGR|  HIREDATE| SAL|COMM|DEPTNO|UPDATED_DATE|
+-----+------+---------+----+----------+----+----+------+------------+
| 7369| SMITH|    CLERK|7902|17-12-1980| 800|null|    20|  2022-01-01|
| 7499| ALLEN| SALESMAN|7698|20-02-1981|1600| 300|    30|  2022-01-01|
| 7521|  WARD| SALESMAN|7698|22-02-1981|1250| 500|    30|  2022-01-01|
| 7566| JONES|  MANAGER|7839|04-02-1981|2975|null|    20|  2022-01-05|
| 7654|MARTIN| SALESMAN|7698|21-09-1981|1250|1400|    30|  2022-01-03|
| 7698|   SGR|  MANAGER|7839|05-01-1981|2850|null|    30|  2022-01-04|
| 7782|  RAVI|  MANAGER|7839|06-09-1981|2450|null|    10|  2022-01-02|
| 7788| SCOTT|  ANALYST|7566|19-04-1987|3000|null|    20|  2022-01-02|
| 7839|  KING|PRESIDENT|null|01-11-1981|5000|null|    10|  2022-01-02|
| 7844|TURNER| SALESMAN|7698|09-08-1981|1500|   0|    30|  2022-01-02|
| 7876| ADAMS|    CLERK|7788|23-05-1987|1100|null|    20|  2022-01-03|
| 7900

## Para cada coluna individual, apenas usar o filter() e count()

In [9]:
df.filter('EMPNO is null').count()

3

## Para todas as colunas:

### Selecionar o número de linhas em cada coluna do df

In [20]:
df.select([count(col(i)) for i in df.columns]).show()

+------------+------------+----------+----------+---------------+----------+-----------+-------------+-------------------+
|count(EMPNO)|count(ENAME)|count(JOB)|count(MGR)|count(HIREDATE)|count(SAL)|count(COMM)|count(DEPTNO)|count(UPDATED_DATE)|
+------------+------------+----------+----------+---------------+----------+-----------+-------------+-------------------+
|          30|          30|        30|        28|             28|        30|         10|           30|                 30|
+------------+------------+----------+----------+---------------+----------+-----------+-------------+-------------------+



### Colocar a condição da coluna ser nula

In [23]:
df.select([count(when(col(i).isNull(), i)) for i in df.columns]).show()

+-----------------------------------------------+-----------------------------------------------+-------------------------------------------+-------------------------------------------+-----------------------------------------------------+-------------------------------------------+---------------------------------------------+-------------------------------------------------+-------------------------------------------------------------+
|count(CASE WHEN (EMPNO IS NULL) THEN EMPNO END)|count(CASE WHEN (ENAME IS NULL) THEN ENAME END)|count(CASE WHEN (JOB IS NULL) THEN JOB END)|count(CASE WHEN (MGR IS NULL) THEN MGR END)|count(CASE WHEN (HIREDATE IS NULL) THEN HIREDATE END)|count(CASE WHEN (SAL IS NULL) THEN SAL END)|count(CASE WHEN (COMM IS NULL) THEN COMM END)|count(CASE WHEN (DEPTNO IS NULL) THEN DEPTNO END)|count(CASE WHEN (UPDATED_DATE IS NULL) THEN UPDATED_DATE END)|
+-----------------------------------------------+-----------------------------------------------+-------------------

### Por conta de ser difícil de observar, pelo tamanho do nome das colunas, colocar alias()

In [26]:
df.select([count(when(col(i).isNull(), i)).alias('NUM Null') for i in df.columns]).show()

+--------+--------+--------+--------+--------+--------+--------+--------+--------+
|NUM Null|NUM Null|NUM Null|NUM Null|NUM Null|NUM Null|NUM Null|NUM Null|NUM Null|
+--------+--------+--------+--------+--------+--------+--------+--------+--------+
|       3|       3|       3|       5|       5|       3|      23|       3|       3|
+--------+--------+--------+--------+--------+--------+--------+--------+--------+

